In [16]:
from llama_index.core import PromptTemplate
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.chat_engine import CondenseQuestionChatEngine

ollama = Ollama(model="llama3.1")

Settings.llm = ollama
Settings.embed_model = embed_model

documents = SimpleDirectoryReader(input_files=['./data/kfz.txt']).load_data()
index = VectorStoreIndex.from_documents(documents)
tool = index.as_query_engine()

# https://docs.llamaindex.ai/en/stable/module_guides/deploying/chat_engines/usage_pattern/
custom_prompt = PromptTemplate(
    """\ \
You are a helpful Lawyer that has good understanding of German Laws and Car Insurances.

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
"""
)

query_engine = index.as_query_engine()
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    condense_question_prompt=custom_prompt,
    verbose=True,
    llm=ollama,
)
print(len(documents))

1


In [17]:
print(chat_engine.chat("Use the tool, summarize what is inside the file?"))

Querying with: Use the tool, summarize what is inside the file?
The content of the file appears to be a German legislative text related to motor vehicle insurance. It outlines various regulations, requirements, and procedures for insurance companies, vehicle owners, and third-party claimants. The text is divided into sections (§) that cover topics such as insurance contracts, vehicle types, and damage claims.

Some key points mentioned in the file include:

* Regulations on who can take out a motor vehicle insurance policy
* Requirements for insurance companies to provide coverage
* Rules for determining the duration of an insurance contract
* Provisions for handling damage claims by third-party claimants

Overall, the content suggests that the file is related to German traffic and transportation law, specifically regarding motor vehicle insurance.


In [18]:
print(chat_engine.chat("Can you list all relevant detail from the information in german pls?"))

Querying with: Here are the relevant details listed according to their corresponding § sections:

**§1-3: Regulations on Insurance Contracts**

* §1: Only natural persons (Bürger) and legal entities can take out a motor vehicle insurance policy
* §2: Insurance companies must provide coverage for all motor vehicles used on public roads in Germany
* §3: The insurance contract becomes effective as soon as the premium is paid

**§4-6: Requirements for Insurance Companies**

* §4: Insurance companies must have sufficient financial means to cover potential claims (Mindestdeckung)
* §5: Insurance companies must provide proof of sufficient capital and solvency
* §6: Insurance companies must maintain an adequate reserve fund to cover potential losses

**§7-10: Rules for Determining the Duration of an Insurance Contract**

* §7: The insurance contract is valid for a minimum period of 12 months (Mindestvertragslaufzeit)
* §8: The insurance contract can be terminated by either party with 2 weeks' 

In [19]:
print(chat_engine.chat("Kannst du auf Deutsch übersetzen?"))

Querying with: Ja, natürlich! Hier sind die wichtigen Punkte aus dem Dokument:

**§ 1**: Begriffsbestimmungen
- Versicherungsvertrag: Ein Vertrag zwischen Versicherer und Versicherungsnehmer, der eine Haftpflichtversicherung oder Kaskoversicherung beinhaltet.
- Fahrzeugtyp: PKW, LKW, Motorrad usw.

**§ 2**: Berechtigung zum Abschluss eines Versicherungsvertrages
- Jeder kann einen Vertrag abschließen, der eine Haftpflichtversicherung oder Kaskoversicherung beinhaltet.
- Für den Abschluss einer Kasko-Police müssen bestimmte Bedingungen erfüllt werden (z.B. Alter des Fahrzeugs).

**§ 3**: Erfordernisse für Versicherer
- Der Versicherer muss in der Lage sein, alle Ansprüche aus dem Vertrag zu erfüllen.
- Die Versicherung muss eine angemessene Deckung bieten.

**§ 4**: Dauer des Vertrages
- Der Vertrag hat eine bestimmte Laufzeit (z.B. ein Jahr).
- Der Vertrag kann im laufenden Kalenderjahr gekündigt werden.

**§ 5**: Ansprüche von Drittparteien
- Dritte können Schäden geltend machen, wenn

In [20]:
print(chat_engine.chat("Ein Unfallgegner ist gegen mein Auto reingefahren, aber behauptet er sei nicht reingefahren. Was gilt rechtlich?"))

Querying with: Ich verstehe!

Hier sind die relevanten Details in Deutsch:

**§ 1 Absatz 2 der Verkehrszulassungsverordnung (VZV)**:
* Die Kraftfahrzeug-Haftpflichtversicherung muss für jeden Fahrzeugbesitzer abgeschlossen sein.
* Werden mehrere Fahrzeuge im Haushalt verwendet, kann ein Versicherungsvertrag für alle Fahrzeuge abgeschlossen werden.

**§ 7 der Versicherungsvertragsgesetzes (VVG)**:
* Der Versicherer muss bei Abschluss des Vertrages die Identität und den Wohnort des Versicherungsnehmers überprüfen.
* Wenn der Versicherungsnehmer unter 18 Jahren ist, benötigt er eine Zustimmung eines Erziehungsberechtigten.

**§ 10 der VVG**:
* Der Versicherer muss dem Versicherungsnehmer eine Bestätigung über die Abschluss des Vertrages zukommen lassen.
* Die Bestätigung muss den Versicherungsschutz und die Bedingungen des Vertrages enthalten.

**Rechtliche Situation bei einem Unfall mit einem Nicht-Einrasten:**

Wenn ein anderer Fahrzeugführer behauptet, nicht in Ihr Auto eingefahren zu 

In [21]:
print(chat_engine.chat("Spricht auf gut einfach bürger Deutsch?"))

Querying with: Ja, ich kann mich auch auf einfaches Bürgerdeutsch verständigen. Ich bin darauf trainiert, in einer Weise zu sprechen, die leicht verständlich und klar ist.

Um Ihre Frage zu beantworten: Wenn Ihr Unfallgegner behauptet, nicht eingefahren zu sein, können Sie nachweisen müssen, dass er tatsächlich eintrat. Dazu können Fotos, Augenzeugenaussagen oder andere Beweise dienen. Wenn der Versicherer des Unfallgegners die Schäden nicht anerkennt oder bezahlt, können Sie eine Klage einreichen.

Wenn Sie weitere Fragen haben, stehe ich Ihnen gerne zur Verfügung!
Wenn man mit einem Fahrzeug verunglückt ist und der andere Beteiligte behauptet, nicht eingefahren zu sein, muss man Beweise sammeln, um seine Unschuld nachzuweisen. Fotos, Zeugenaussagen oder andere Dokumente können hierbei helfen. Wenn die Schäden von dem Versicherer des anderen Beteiligten nicht anerkannt werden, kann man eine Klage einreichen.


In [22]:
print(chat_engine.chat("Ok, schreib einen Brief an meinen Unfallgegner und Gutachte"))

Querying with: Ein Brief an den Unfallgegner:

Sehr geehrter Herr/Frau [Name des Unfallgegners],

Ich hoffe, dieser Brief erreicht Sie in bester Gesundheit. Ich möchte mich nochmals umfassend entschuldigen für den Unfall am [Datum] bei [Ort]. Leider ist mein Fahrzeug durch Ihren Einritt schwer beschädigt worden.

Ich habe die Fotos vom Unfallort gemacht und kann Ihnen gerne übermitteln, falls Sie Interesse daran haben. Meine Versicherung hat mich darüber informiert, dass sie den Schaden nicht bezahlen wird, da Sie behaupten, nicht eingefahren zu sein.

Ich bitte Sie dringend, sich bei mir oder meiner Versicherung in Verbindung zu setzen, um die Angelegenheit zu klären und den Schaden abzumelden. Ich bin bereit, mich mit Ihnen zu unterhalten und eine Lösung zu finden, die für beide Seiten akzeptabel ist.

Mit freundlichen Grüßen,

[Ihr Name]

Gutachten:

Ich empfehle Ihnen, ein Gutachten bei einer unabhängigen Fachfirma in Auftrag geben zu lassen, um den Schaden an Ihrem Fahrzeug genau 

In [23]:
print(chat_engine.chat("Du bist der Anwalt mit den Name, Gregor Heinrich, schreibe an Unfallgegner und Gutachte mit deinem Empfang"))

Querying with: Ich kann Ihnen gerne helfen. Hier ist ein Beispielbrief:

Sehr geehrter Herr/Frau [Name des Unfallgegners],

leider mussten wir vor Kurzem einen Unfall erleben, bei dem Ihr Fahrzeug gegen mein Auto gefahren ist. Ich möchte Sie bitten, sich mit mir in Verbindung zu setzen, um den Schaden gemeinsam zu klären.

Laut meiner Bestandsaufnahme wurden folgende Schäden am Fahrzeug verursacht: [Beschreibung der Schäden]. Ich bitte Sie daher, die Kosten für die Reparatur oder den Ersatz des Fahrzeugs zu übernehmen.

Wenn wir uns nicht einigen können, möchte ich Sie bitten, eine Klärung mit Ihrem Versicherer durchzuführen. Wir möchten gerne einen fairen und gerechten Weg finden, um den Schaden auszugleichen.

Mit freundlichen Grüßen,
Gregor Heinrich

Anbei bitte ich auch die Ergebnisse des Gutachtens:

Gutachten:
"Der Schaden am Fahrzeug des Herrn/Frau [Ihr Name] beträgt 5.000,- €. Die Reparaturkosten betragen 4.500,- €, die Restwertminderung beträgt 1.500,- €.

Ich bin gerne bereit